# inferencer

> Infers the schedule
- order: 9

In [ ]:
#| default_exp inferencer

In [ ]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# from collections import namedtuple, defaultdict
# import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
# import time
# import math
# from pprint import pprint
## !pip install -U "ray"
# import ray
# import json

import AIShiftScheduler_KE5.config as cf
from AIShiftScheduler_KE5.model import *
from AIShiftScheduler_KE5.policy import *


len(RESOURCE_TYPES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(TYPES)=13
['Manager', 'AssistMngr', 'AssistMngr', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc']
len(aNAMES)=13
['Manager_Matt', 'AssistMngr_Mike', 'AssistMngr_Tanner', 'RetailAssoc_Jake', 'RetailAssoc_James', 'RetailAssoc_Jane', 'RetailAssoc_John', 'RetailAssoc_Jim', 'RetailAssoc_Jenny', 'RetailAssoc_Jeremy', 'RetailAssoc_Judy', 'RetailAssoc_Julie', 'RetailAssoc_Jeffrey']

len(bNAMES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(abNAMES)=13
['Manager_Matt___Manager', 'AssistMngr_Mike___AssistMngr', 'AssistMngr_Tanner___AssistMngr', 'RetailAssoc_Jake___RetailAssoc', 'RetailAssoc_James___RetailAssoc', 'RetailAssoc_Jane___RetailAssoc', 'RetailAssoc_John___RetailAssoc', 'RetailAssoc_Jim___RetailAssoc', 'RetailAssoc_Jenny___RetailAssoc', 'RetailAssoc_Jeremy___RetailAssoc', 'RetailAssoc_Judy___RetailAssoc', 'RetailAsso

In [ ]:
# pd.__version__

In [ ]:
# !pip freeze | grep pandas
# !pip freeze|grep openpyxl
# !pip freeze

In [ ]:
# !pip install -U ray==2.8.1
# !pip install openpyxl

In [ ]:
# #| export
# pd.options.display.float_format = '{:,.4f}'.format
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
# #| export
# base_dir = '.'
# file_name = 'shift_scheduler_data.xlsx'

In [ ]:
#| export
def get_best_theta_Alloc():
  best_theta_Alloc = \
    Policy(None).build_theta({
      'thCumSlots': cf.BEST_THETA_Alloc[0],
      'thSickProb': cf.BEST_THETA_Alloc[1],
      'thCumMerits': cf.BEST_THETA_Alloc[2],
      'thContSlots': cf.BEST_THETA_Alloc[3],
      'thSelect': cf.BEST_THETA_Alloc[4]
    })
  return best_theta_Alloc

In [ ]:
#| export
def do_infer(L, T, best_theta_Alloc):
  # global DEM, MER, \
  # global ThetaStar_expCbarcum_Alloc_evalu_opt, ThetaStar_expCtilcum_Alloc_evalu_opt, \
  # Best_theta_Alloc_evalu_opt, Worst_theta_Alloc_evalu_opt, \
  # Record_Alloc_evalu_opt, Df_Alloc_evalu_opt

  M = Model()
  P = Policy(M)
  ## DEM = DemandSimulator(seed=SEED_EVALU)
  ## MER = MeritSimulator(seed=SEED_EVALU)
  # thetasOpt = []; thetasOpt.append(get_best_theta_Alloc()) ##from storage
  thetasOpt = []; thetasOpt.append(best_theta_Alloc) ##from storage
  # ThetaStar_expCbarcum_Alloc_evalu_opt, ThetaStar_expCtilcum_Alloc_evalu_opt, \
  # _, _, \
  # Best_theta_Alloc_evalu_opt, Worst_theta_Alloc_evalu_opt, \
  # _, _, \
  # _, _, \
  # Record_Alloc_evalu_opt = \
  #  P.perform_grid_search_sample_paths('X__Alloc', thetasOpt, L, T)
  return \
    P.perform_grid_search_sample_paths('X__Alloc', thetasOpt, L, T)
    
  # Df_Alloc_evalu_opt = pd.DataFrame.from_records(
  #     Record_Alloc_evalu_opt[:First_n_t], columns=Labels)
  # print(
  #   f'{ThetaStar_expCbarcum_Alloc_evalu_opt.iloc[-1]=:.2f}')

In [ ]:
#| export
from io import StringIO
def prepare_schedule_shifts(df, buf):
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  buf.write(f"SCHEDULE SHIFTS:\n")
  buf.write(f"================\n")
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    buf.write(f'\n************** {resName}:\n')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ts_1 = sched_list[0]
      dow_1 = sched_list[0].day_of_week
      buf.write(f"{(sched_list[0]-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1:
          buf.write(f"{(ts_1).strftime('%a %b %d %Hh%M')}\n\n")
          buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
        dow_1 = dow
        ts_1 = ts
      buf.write(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}\n")
  buf.write(f'\n{cf.CONTIGUOUS_REWARD=}\n')
  buf.write(f'{cf.MAX_DAILY_SLOT_RUN=} ({cf.RESOLUTION}s)\n')
  buf.write(f'{cf.TH_CumSlots_SPEC=}\n')
  buf.write(f'{cf.TH_SickProb_SPEC=}\n')
  buf.write(f'{cf.TH_CumMerits_SPEC=}\n')
  buf.write(f'{cf.TH_ContSlots_SPEC=}\n')
  buf.write(f'{cf.TH_Select_SPEC=}\n')
  return buf.getvalue()

In [ ]:
#| export
from io import StringIO
def prepare_schedule_slots(df, buf):
  gap_mins = gap_minutes(cf.RESOLUTION)
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  buf.write(f"SCHEDULE SLOTS:\n")
  buf.write(f"===============\n")
  n_gaps = 0
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    buf.write(f'\n************** {resName}:\n')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ## print each slot and indicate gaps
      ts_1 = sched_list[0]
      dow_1 = -1
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1: buf.write('\n')
        if (int((ts.to_datetime64() - ts_1.to_datetime64())/(1e9*60)) > gap_mins)\
          and (dow == dow_1):
          n_gaps += 1
          buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP\n")
        else:
          buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
        dow_1 = dow
        ts_1 = ts
  buf.write(f'\nTOTAL NUMBER OF GAPS FOR ALL RESOURCES: {n_gaps}\n')
  buf.write(f'{cf.CONTIGUOUS_REWARD=}\n')
  buf.write(f'{cf.MAX_DAILY_SLOT_RUN=} ({cf.RESOLUTION}s)\n')
  buf.write(f'{cf.TH_CumSlots_SPEC=}\n')
  buf.write(f'{cf.TH_SickProb_SPEC=}\n')
  buf.write(f'{cf.TH_CumMerits_SPEC=}\n')
  buf.write(f'{cf.TH_ContSlots_SPEC=}\n')
  buf.write(f'{cf.TH_Select_SPEC=}\n')
  return buf.getvalue()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()